In [1]:
NUMVBER_OF_PASSAGES = 20
PATH_QUESTION = 'finetune_qa_pairs_14k/'
PATH_SAVE_QUESTIONS_BM25 = 'finetune_qa_pairs_14k_sem_retrieval/' #mudar path
INDEX_KNOWLEDGE_BASE = ["wiki_news_ambiente"]

In [2]:
#from haystack.preprocessor.cleaning import clean_wiki_text
#from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

06/15/2021 19:22:56 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/15/2021 19:22:56 - INFO - faiss.loader -   Loading faiss.
06/15/2021 19:22:56 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
#! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
#! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
#! chown -R daemon:daemon elasticsearch-7.9.2
##! chmod 755 -R elasticsearch-7.9.2/bin/elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [4]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="")

06/15/2021 19:23:27 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.057s]
06/15/2021 19:23:27 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.006s]
06/15/2021 19:23:27 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.002s]
06/15/2021 19:23:27 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.015s]
06/15/2021 19:23:27 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


In [5]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [6]:
from haystack.pipeline import Pipeline

extractive_pipeline = Pipeline()
extractive_pipeline.add_node(component=retriever, name="ESRetriever1", inputs=["Query"])

In [7]:
res = extractive_pipeline.run(query="Copan", top_k_retriever=5, index = INDEX_KNOWLEDGE_BASE)#, index = "teste")
print(res)

06/15/2021 19:23:27 - WARNING - elasticsearch -   POST http://localhost:9200/wiki_news/_search [status:404 request:0.013s]


Exception: Exception while running node `ESRetriever1` with input `{'pipeline_type': 'Query', 'query': 'Copan', 'top_k_retriever': 5, 'index': ['wiki_news']}`: NotFoundError(404, 'index_not_found_exception', 'no such index [wiki_news]', wiki_news, index_or_alias), full stack trace: Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/haystack/pipeline.py", line 126, in run
    node_output, stream_id = self.graph.nodes[node_id]["component"].run(**node_input)
  File "/usr/local/lib/python3.8/dist-packages/haystack/retriever/base.py", line 180, in run
    output, stream = run_query_timed(**kwargs)
  File "/usr/local/lib/python3.8/dist-packages/haystack/retriever/base.py", line 43, in wrapper
    ret = fn(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/haystack/retriever/base.py", line 197, in run_query
    documents = self.retrieve(query=query, filters=filters, top_k=top_k_retriever, index=index)
  File "/usr/local/lib/python3.8/dist-packages/h

In [ ]:

import pandas as pd
train_source = pd.read_csv(PATH_QUESTION + 'train.source', sep = "\t", header = None).values.tolist()  

train_target = pd.read_csv(PATH_QUESTION + 'train.target', sep = "\t", header = None).values.tolist()  

test_source = pd.read_csv(PATH_QUESTION + 'test.source', sep = "\t", header = None).values.tolist()  

test_target = pd.read_csv(PATH_QUESTION + 'test.target', sep = "\t", header = None).values.tolist()   

val_source = pd.read_csv(PATH_QUESTION + 'val.source', sep = "\t", header = None).values.tolist()  

val_target = pd.read_csv(PATH_QUESTION + 'val.target', sep = "\t", header = None).values.tolist()  


In [ ]:
train_docs = [] 
for i in range(len(train_source)):
    res = extractive_pipeline.run(query=train_source[i][0], top_k_retriever=20, index = INDEX_KNOWLEDGE_BASE)
    train_docs.append({"question":train_source[i], "answer": train_target[i], "documents":res["documents"]})
    if i %100 == 0:
        print(i)

In [ ]:
val_docs = [] 
for i in range(len(val_source)):
    res = extractive_pipeline.run(query=test_source[i][0], top_k_retriever=20, index = INDEX_KNOWLEDGE_BASE)
    val_docs.append({"question":val_source[i], "answer": val_target[i], "documents":res["documents"]})
    if i %100 == 0:
        print(i)

In [ ]:
test_docs = [] 
for i in range(len(test_source)):
    res = extractive_pipeline.run(query=test_source[i][0], top_k_retriever=20, index = INDEX_KNOWLEDGE_BASE)
    test_docs.append({"question":test_source[i], "answer": test_target[i], "documents":res["documents"]})
    if i %100 == 0:
        print(i)

In [ ]:
import pickle

file_to_write = open(PATH_SAVE_QUESTIONS_BM25 + "train_doc.pickle", "wb")

pickle.dump(train_docs, file_to_write)
file_to_write.close()


In [ ]:
file_to_write = open(PATH_SAVE_QUESTIONS_BM25 + "val_doc.pickle", "wb")

pickle.dump(val_docs, file_to_write)
file_to_write.close()

In [ ]:
file_to_write = open(PATH_SAVE_QUESTIONS_BM25 + "test_doc.pickle", "wb")

pickle.dump(test_docs, file_to_write)
file_to_write.close()